In [7]:
code = """
class Company:
    def __init__(self, name, revenue, ebitda, ebit_margin, wacc, shares_outstanding):
        self.name = name
        self.revenue = revenue
        self.ebitda = ebitda
        self.ebit_margin = ebit_margin
        self.wacc = wacc
        self.shares_outstanding = shares_outstanding

    def __repr__(self):
        return (f"{self.name} - Revenue: {self.revenue}M, EBITDA: {self.ebitda}M, "
                f"EBITDA Margin: {self.ebit_margin*100:.1f}%, WACC: {self.wacc*100:.1f}%, "
                f"Shares: {self.shares_outstanding}M")
"""
with open("company.py", "w") as f:
    f.write(code)


In [6]:
code = """
import yfinance as yf

def get_company_data(ticker, default_wacc=0.08):
    stock = yf.Ticker(ticker)
    info = stock.info

    try:
        name = info['longName']
        revenue = info['totalRevenue'] / 1e6
        ebitda = info['ebitda'] / 1e6
        shares_outstanding = info['sharesOutstanding'] / 1e6

        ebitda_margin = ebitda / revenue
        wacc = default_wacc

        return {
            "name": name,
            "revenue": round(revenue, 2),
            "ebitda": round(ebitda, 2),
            "ebitda_margin": round(ebitda_margin, 4),
            "wacc": round(wacc, 4),
            "shares_outstanding": round(shares_outstanding, 2)
        }

    except Exception as e:
        print(f"Erreur de récupération des données pour {ticker} : {e}")
        return None
"""

with open("company_loader.py", "w") as f:
    f.write(code)

In [12]:
code = """
import pandas as pd

def load_synergy_benchmarks(file_path="/Users/romaincalfati/Documents/HEC M1/Finance application/Python Stuff/mna_deals.xlsx"):
    df = pd.read_excel(file_path)

    # Calcul des médianes
    benchmarks = {
        "cost_synergy_pct": df["Cost Synergy %"].median() / 100,
        "revenue_synergy_pct": df["Revenue Synergy %"].median() / 100,
        "synergy_ebit_margin": df["EBIT Margin %"].median() / 100,
        "integration_cost": df["Integration Cost ($M)"].median(),
        "duration_years": int(df["Duration (years)"].median())
    }

    return benchmarks
"""

with open("synergy_loader.py", "w") as f:
    f.write(code)

In [1]:
from company_loader import get_company_data
from company import Company

# Côté acquéreur
data = get_company_data("AAPL")  # Apple

if data:
    acquirer = Company(
        name=data["name"],
        revenue=data["revenue"],
        ebitda=data["ebitda"],
        ebit_margin=data["ebitda_margin"],
        wacc=data["wacc"],
        shares_outstanding=data["shares_outstanding"]
    )

# Côté cible
data = get_company_data("MSFT")  # Microsoft

if data:
    target = Company(
        name=data["name"],
        revenue=data["revenue"],
        ebitda=data["ebitda"],
        ebit_margin=data["ebitda_margin"],
        wacc=data["wacc"],
        shares_outstanding=data["shares_outstanding"]
    )

print(acquirer)
print(target)

Apple Inc. - Revenue: 400366.01M, EBITDA: 138866.0M, EBITDA Margin: 34.7%, WACC: 8.0%, Shares: 14935.8M
Microsoft Corporation - Revenue: 270010.01M, EBITDA: 149172.99M, EBITDA Margin: 55.2%, WACC: 8.0%, Shares: 7432.54M


In [14]:
class SynergyModel:
    def __init__(
        self, 
        acquirer, 
        target, 
        cost_synergy_pct,           # ex: 5% = 0.05
        revenue_synergy_pct,        # ex: 3% = 0.03
        synergy_ebit_margin,        # Marge EBIT estimée pour revenus synergiques (ex: 20%)
        integration_cost,           # Montant en millions
        duration_years
    ):
        self.acquirer = acquirer
        self.target = target
        self.cost_synergy_pct = cost_synergy_pct
        self.revenue_synergy_pct = revenue_synergy_pct
        self.synergy_ebit_margin = synergy_ebit_margin
        self.integration_cost = integration_cost
        self.duration_years = duration_years # Nbr d'années sur lesquelles les synergies sont réalisées
        self.discount_rate = acquirer.wacc  # WACC de l’acquéreur pour l’actualisation

    def discounted_value(self, amount):
        return sum(amount / ((1 + self.discount_rate) ** t) for t in range(1, self.duration_years + 1))

    def compute(self):
        # Données combinées
        combined_revenue = self.acquirer.revenue + self.target.revenue
        combined_costs = (
            self.acquirer.revenue * (1 - self.acquirer.ebit_margin) +
            self.target.revenue * (1 - self.target.ebit_margin)
        )

        # Synergies calculées à partir des %
        annual_cost_synergies = self.cost_synergy_pct * combined_costs
        annual_revenue_synergies = self.revenue_synergy_pct * combined_revenue * self.synergy_ebit_margin

        # Actualisation
        pv_cost_synergies = self.discounted_value(annual_cost_synergies)
        pv_revenue_synergies = self.discounted_value(annual_revenue_synergies)
        pv_integration_cost = self.discounted_value(self.integration_cost)

        total_synergies = pv_cost_synergies + pv_revenue_synergies
        net_value_created = total_synergies - pv_integration_cost

        return {
            "Annual Cost Synergies": round(annual_cost_synergies, 2),
            "Annual Revenue Synergies (EBIT)": round(annual_revenue_synergies, 2),
            "PV Cost Synergies": round(pv_cost_synergies, 2),
            "PV Revenue Synergies": round(pv_revenue_synergies, 2),
            "PV Integration Cost": round(pv_integration_cost, 2),
            "Net Value Created": round(net_value_created, 2)
        }

    def compute_interval(self, delta_cost=0.01, delta_rev=0.01, delta_margin=0.02):
            """
            Calcule une fourchette [min, max] de la valeur nette créée.
            """
            scenarios = []
    
            # Crée des combinaisons de variations
            for dc in [-delta_cost, 0, delta_cost]:
                for dr in [-delta_rev, 0, delta_rev]:
                    for dm in [-delta_margin, 0, delta_margin]:
                        model = SynergyModel(
                            acquirer=self.acquirer,
                            target=self.target,
                            cost_synergy_pct=self.cost_synergy_pct + dc,
                            revenue_synergy_pct=self.revenue_synergy_pct + dr,
                            synergy_ebit_margin=self.synergy_ebit_margin + dm,
                            integration_cost=self.integration_cost,
                            duration_years=self.duration_years
                        )
                        value = model.compute()["Net Value Created"]
                        scenarios.append(value)
    
            return min(scenarios), max(scenarios)

In [22]:
from synergy_loader import load_synergy_benchmarks

benchmarks = load_synergy_benchmarks("/Users/romaincalfati/Documents/HEC M1/Finance application/Python Stuff/mna_deals.xlsx")

model = SynergyModel(
    acquirer=acquirer,
    target=target,
    cost_synergy_pct=benchmarks["cost_synergy_pct"],
    revenue_synergy_pct=benchmarks["revenue_synergy_pct"],
    synergy_ebit_margin=benchmarks["synergy_ebit_margin"],
    integration_cost=benchmarks["integration_cost"],
    duration_years=benchmarks["duration_years"]
)

result = model.compute()
low, high = model.compute_interval()

for k, v in result.items():
    print(f"{k}: {v}M")

print("------------------")
print(f"Range of Value Creation (sensitivity): {low:.2f}M – {high:.2f}M")

Annual Cost Synergies: 19308.6M
Annual Revenue Synergies (EBIT): 4489.84M
PV Cost Synergies: 63952.54M
PV Revenue Synergies: 14870.93M
PV Integration Cost: 2152.88M
Net Value Created: 76670.59M
------------------
Range of Value Creation (sensitivity): 57967.31M – 96262.01M
